In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import SimpleRCNN as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'simplercnn')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'simplercnn_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')
RESULT_CSV = os.path.join(RESULTS_PATH, 'simplercnn.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 230s 2ms/step - loss: 0.2770 - acc: 0.9214 - val_loss: 0.3973 - val_acc: 0.8745


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.53683, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_0.h5
Epoch 2/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.2174 - acc: 0.9381 - val_loss: 0.3366 - val_acc: 0.9171

Epoch 00002: _f1_monitor did not improve from 0.53683
Epoch 3/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.2018 - acc: 0.9422 - val_loss: 0.3598 - val_acc: 0.8955

Epoch 00003: _f1_monitor improved from 0.53683 to 0.57168, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_0.h5
Epoch 4/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.1914 - acc: 0.9440 - val_loss: 0.2648 - val_acc: 0.9310

Epoch 00004: _f1_monitor improved from 0.57168 to 0.58549, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_0.

105000/105000 [==============================] - 228s 2ms/step - loss: 0.1232 - acc: 0.9604 - val_loss: 0.3723 - val_acc: 0.9023

Epoch 00008: _f1_monitor did not improve from 0.67017
Epoch 9/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.1085 - acc: 0.9654 - val_loss: 0.4045 - val_acc: 0.8933

Epoch 00009: _f1_monitor did not improve from 0.67017

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 2th grain: f1_score - 0.6701738994209958, acc - 0.9023333333333333
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 229s 2ms/step - loss: 0.3737 - acc: 0.8828 - val_loss: 0.3642 - val_acc: 0.8929

Epoch 00001: _f1_monitor improved from -inf to 0.37896, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_3.h5
Epoch 2/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.3069 - acc: 0.902

Epoch 9/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.0259 - acc: 0.9928 - val_loss: 0.1342 - val_acc: 0.9715

Epoch 00009: _f1_monitor did not improve from 0.68808
Epoch 10/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.0217 - acc: 0.9938 - val_loss: 0.1391 - val_acc: 0.9710

Epoch 00010: _f1_monitor did not improve from 0.68808

Epoch 00010: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 5th grain: f1_score - 0.6880758123987492, acc - 0.9718
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 229s 2ms/step - loss: 0.3488 - acc: 0.8937 - val_loss: 0.2720 - val_acc: 0.9213

Epoch 00001: _f1_monitor improved from -inf to 0.63860, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_6.h5
Epoch 2/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.2639 - acc: 0.92

105000/105000 [==============================] - 228s 2ms/step - loss: 0.4131 - acc: 0.8451 - val_loss: 0.9406 - val_acc: 0.7109

Epoch 00004: _f1_monitor improved from 0.34189 to 0.37883, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_9.h5
Epoch 5/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.3798 - acc: 0.8605 - val_loss: 0.7307 - val_acc: 0.7647

Epoch 00005: _f1_monitor improved from 0.37883 to 0.44827, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_9.h5
Epoch 6/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.3438 - acc: 0.8762 - val_loss: 0.5963 - val_acc: 0.8188

Epoch 00006: _f1_monitor improved from 0.44827 to 0.56075, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_9.h5
Epoch 7/100
105000/105000 [==============================] - 

105000/105000 [==============================] - 230s 2ms/step - loss: 0.6732 - acc: 0.7456 - val_loss: 0.6375 - val_acc: 0.7599

Epoch 00001: _f1_monitor improved from -inf to 0.56105, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_12.h5
Epoch 2/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.5415 - acc: 0.7990 - val_loss: 0.5785 - val_acc: 0.7817

Epoch 00002: _f1_monitor improved from 0.56105 to 0.67398, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_12.h5
Epoch 3/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.5012 - acc: 0.8143 - val_loss: 0.5623 - val_acc: 0.8041

Epoch 00003: _f1_monitor did not improve from 0.67398
Epoch 4/100
105000/105000 [==============================] - 229s 2ms/step - loss: 0.4638 - acc: 0.8296 - val_loss: 0.9269 - val_acc: 0.6741

Epoch 00004: _f1_monitor did not improve 

Epoch 5/100
105000/105000 [==============================] - 229s 2ms/step - loss: 0.4683 - acc: 0.8104 - val_loss: 0.6667 - val_acc: 0.7360

Epoch 00005: _f1_monitor improved from 0.58251 to 0.58953, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_15.h5
Epoch 6/100
105000/105000 [==============================] - 229s 2ms/step - loss: 0.4220 - acc: 0.8328 - val_loss: 0.6790 - val_acc: 0.7364

Epoch 00006: _f1_monitor improved from 0.58953 to 0.60035, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_15.h5
Epoch 7/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.3690 - acc: 0.8583 - val_loss: 0.8705 - val_acc: 0.7011

Epoch 00007: _f1_monitor improved from 0.60035 to 0.60395, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_15.h5
Epoch 8/100
105000/105000 [===================

105000/105000 [==============================] - 229s 2ms/step - loss: 0.4680 - acc: 0.8283 - val_loss: 0.6529 - val_acc: 0.7840

Epoch 00004: _f1_monitor did not improve from 0.52820
Epoch 5/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.4341 - acc: 0.8432 - val_loss: 0.6724 - val_acc: 0.7521

Epoch 00005: _f1_monitor improved from 0.52820 to 0.54840, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/simplercnn/simplercnn_18.h5
Epoch 6/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.4004 - acc: 0.8591 - val_loss: 0.6039 - val_acc: 0.7851

Epoch 00006: _f1_monitor did not improve from 0.54840
Epoch 7/100
105000/105000 [==============================] - 228s 2ms/step - loss: 0.3602 - acc: 0.8750 - val_loss: 0.9329 - val_acc: 0.7482

Epoch 00007: _f1_monitor did not improve from 0.54840
Epoch 8/100
105000/105000 [==============================] - 229s 2ms/step - loss: 0.3199 - acc: 0.8899 - 

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.612761002485214
The 1th f1: 0.4825772990026381
The 2th f1: 0.6701738994209958
The 3th f1: 0.5664023691357214
The 4th f1: 0.7421149514323554
The 5th f1: 0.6880758123987492
The 6th f1: 0.6696770918790219
The 7th f1: 0.7343245581441576
The 8th f1: 0.6314188827415947
The 9th f1: 0.5895586450888557
The 10th f1: 0.6358883037021885
The 11th f1: 0.6740021353726983
The 12th f1: 0.6739833842010684
The 13th f1: 0.6550640841376186
The 14th f1: 0.6345467293400083
The 15th f1: 0.6242166062827406
The 16th f1: 0.508251835638878
The 17th f1: 0.694317488160731
The 18th f1: 0.5483999520434892
The 19th f1: 0.6266770988034771
The average f1 of val is 0.6331216064706101


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)